<a href="https://colab.research.google.com/github/olanigan/DSPy_Cookbook/blob/main/dspy_3_0_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!pip install -U dspy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.8 MB/s eta 0:00:00


## Setup

In [2]:
import dspy
from google.colab import userdata
api_key= userdata.get('NGEMINI_API_KEY')
lm = dspy.LM("gemini/gemini-2.5-flash", api_key=api_key)
dspy.configure(lm=lm)

## LM Test

In [3]:
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

['This is a test!']

## Signature - Classification

In [6]:

from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='neutral',
    confidence=0.85
)

In [7]:
## Changing the text slightly
classify(sentence="This book was super fun to read, especially the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.98
)

## Signature - Information Extraction

In [10]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    facts: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.facts)
print(response.entities)

Apple Announces iPhone 14
['Apple Inc. announced the iPhone 14.', 'The announcement was made today.', 'Tim Cook, CEO of Apple Inc., highlighted the new features of the iPhone 14.', 'The features were highlighted in a press release.']
[{'name': 'Apple Inc.', 'type': 'Organization'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person', 'role': 'CEO'}]


## Modules - RAG w/ COT

In [16]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

rag = dspy.ChainOfThought("context, question -> response")

question = "What's the name of the castle that David Gregory inherited?"
context = search_wikipedia(question)

#print retrieved context using pprint
import pprint
pprint.pprint(context)

rag(context=context, question=question)

['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a '
 'Scottish physician and inventor. His surname is sometimes spelt as Gregorie, '
 'the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three '
 'of his twenty-nine children became mathematics professors. He is credited '
 'with inventing a military cannon that Isaac Newton described as "being '
 'destructive to the human species". Copies and details of the model no longer '
 "exist. Gregory's use of a barometer to predict farming-related weather "
 'conditions led him to be accused of witchcraft by Presbyterian ministers '
 'from Aberdeen, although he was never convicted.',
 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης '
 ', Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" '
 'and the long-reigning catepan of Italy from 998 to 1006. In December 999, '
 'and again on February 2, 1002, he reinstituted and confirmed the possessions '
 'of

Prediction(
    reasoning='The question asks for the name of the castle inherited by David Gregory. I will look for information about David Gregory in the provided context and identify any mention of a castle he inherited.\n\nContext [1] states: "David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. ... He inherited Kinnairdy Castle in 1664."\n\nThis directly answers the question.',
    response='Kinnairdy Castle'
)

## Modules - ReAct

In [17]:

def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 45 times by the number of times Chelsea has won the Champions League?")
print(pred.answer)

90.0


In [26]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (float):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}        # note: the value you produce must be a single float value\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.'},
  {'role': 'user',
   'content': '[[ ## question ## ]]\nWhat is 45 times by the number of times Chelsea has won the Champions League?\n\n[[ ## trajectory ## ]]\n[[ ## thought_0 ## ]]\nThe user is asking to multiply 45 by the number of times Chelsea has won the Champions League. I need to find out how many times Chelsea has won the Cham